# Práctica tipo laboratorio: Introducción a ROOT y PyROOT

En esta práctica trabajaremos con el framework **ROOT** (desarrollado por el CERN) a través de **PyROOT**, utilizando Python.

La práctica está organizada en ejercicios graduales que cubren:

1. Creación y visualización de histogramas.
2. Manejo de archivos ROOT (`TFile`).
3. Creación y lectura de árboles (`TTree`).
4. Uso de `RDataFrame` para análisis de datos.
5. Generación de histogramas 2D y almacenamiento de resultados.

Cada sección incluye:
- Una **descripción del objetivo**.
- Indicaciones generales.
- Celdas de código base donde podrás completar la solución.


## Objetivos de la práctica

Al finalizar esta práctica, el estudiante será capaz de:

- Manejar los objetos básicos de ROOT: `TFile`, `TH1`, `TH2`, `TTree`.
- Crear, llenar y visualizar histogramas de una o más variables.
- Construir y recorrer un `TTree` con datos simulados.
- Guardar resultados de análisis en archivos ROOT para uso posterior.
- Utilizar `RDataFrame` como interfaz de alto nivel para análisis de datos en ROOT.


## Requisitos previos

- Conocimientos básicos de Python.
- ROOT instalado con soporte para PyROOT (`import ROOT` debe funcionar).
  - En Google Colab, esto requiere una instalación previa (no incluida en este notebook).
- Conocimientos básicos de probabilidad y estadística (distribuciones, media, sigma).

> **Nota:** Antes de comenzar los ejercicios, verifica que el siguiente bloque puede importar ROOT sin errores.


In [ ]:
%%bash

source /content/root/bin/thisroot.sh
python3.10 - << 'EOF'
import ROOT
print("Versión de ROOT:", ROOT.gROOT.GetVersion())
EOF

---

## Ejercicio 1: Crear y dibujar un histograma simple

**Objetivo:** Familiarizarse con la clase `TH1F` y el flujo básico de creación, llenado y visualización de un histograma.

### Instrucciones

1. Crea un histograma de tipo `TH1F` con:
   - Nombre interno: `"h1"`
   - Título visible: `"Distribución gaussiana; x; Eventos"`
   - 100 bins, en el rango de \(-5, 5\).
2. Llena el histograma con 10 000 números aleatorios distribuidos de forma gaussiana (usa `ROOT.gRandom.Gaus()`).
3. Crea un `TCanvas` y dibuja el histograma.

Completa el siguiente bloque de código donde se indica.


In [ ]:
# Ejercicio 1

# 1. Crear histograma TH1F
h1 = ROOT.TH1F("h1", "Distribucion gaussiana; x; Eventos", 100, -5, 5)

# 2. Llenar el histograma con 10 000 valores gaussianos
for i in range(10000):
    x = ROOT.gRandom.Gaus(0, 1)  # media=0, sigma=1
    h1.Fill(x)

# 3. Dibujar el histograma en un canvas
c1 = ROOT.TCanvas("c1", "Ejercicio 1", 800, 600)
h1.Draw()
c1.Draw()

---

## Ejercicio 2: Personalizar el estilo del histograma

**Objetivo:** Aprender a modificar el estilo de un histograma en ROOT.

### Instrucciones

A partir del histograma del **Ejercicio 1** (`h1`):

1. Cambia el color de la línea.
2. Cambia el estilo de línea (por ejemplo, continua, punteada).
3. Cambia el tipo de marcador y su tamaño.
4. Ajusta el título del eje X y del eje Y para que sean descriptivos.

Puedes reutilizar el mismo histograma o crear uno nuevo. Usa `SetLineColor`, `SetLineStyle`, `SetMarkerStyle`, `SetMarkerSize`, `GetXaxis().SetTitle()`, etc.


In [ ]:
# Ejercicio 2

# Puedes reutilizar h1 o crear un nuevo histograma.
# Aquí se asume que h1 ya existe (del ejercicio 1).

h1.SetLineColor(ROOT.kBlue)
h1.SetLineStyle(1)
h1.SetMarkerStyle(20)
h1.SetMarkerSize(0.8)

h1.GetXaxis().SetTitle("Variable x [unidades arbitrarias]")
h1.GetYaxis().SetTitle("Numero de eventos")

c2 = ROOT.TCanvas("c2", "Ejercicio 2 - Estilo", 800, 600)
h1.Draw("HIST")
c2.Draw()

---

## Ejercicio 3: Crear un `TTree` mínimo y guardarlo en un archivo ROOT

**Objetivo:** Practicar la creación de un `TTree` con dos variables correlacionadas y el uso de `TFile` para guardar datos en disco.

### Instrucciones

1. Crea un archivo `TFile` llamado `"tree_ejemplo.root"` en modo `"RECREATE"`.
2. Crea un `TTree` llamado `"arbol"` con dos ramas tipo `float`:
   - `x`
   - `y` (por ejemplo, `y = x + ruido_gaussiano`)
3. Llena el árbol con 5000 eventos.
4. Guarda el árbol en el archivo y cierra el archivo.

Utiliza la clase `array` de Python para declarar variables por referencia.


In [ ]:
# Ejercicio 3

from array import array
import random

# 1. Crear archivo ROOT
f_out = ROOT.TFile("tree_ejemplo.root", "RECREATE")

# 2. Crear TTree y ramas
tree = ROOT.TTree("arbol", "Arbol de ejemplo")

x = array('f', [0.])
y = array('f', [0.])

tree.Branch("x", x, "x/F")
tree.Branch("y", y, "y/F")

# 3. Llenar el arbol con 5000 eventos
for i in range(5000):
    x[0] = random.gauss(0, 1)
    y[0] = x[0] + random.gauss(0, 0.5)
    tree.Fill()

# 4. Escribir y cerrar el archivo
tree.Write()
f_out.Close()

print("Archivo 'tree_ejemplo.root' creado con el TTree 'arbol'.")

---

## Ejercicio 4: Leer el `TTree` y explorar distribuciones

**Objetivo:** Practicar la lectura de un archivo ROOT y el uso del árbol para generar histogramas y gráficas de correlación.

### Instrucciones

1. Abre el archivo `tree_ejemplo.root` creado en el ejercicio anterior.
2. Obtén el `TTree` `"arbol"`.
3. Construye:
   - Un histograma de `x`.
   - Un histograma de `y`.
4. Opcional: crea un `TGraph` de `y` vs `x` para visualizar la correlación.

Puedes usar un bucle `for entry in tree:` o bien `tree.Draw("x>>histoX")`.


In [ ]:
# Ejercicio 4

# 1. Abrir archivo y obtener TTree
f_in = ROOT.TFile("tree_ejemplo.root", "READ")
arbol = f_in.Get("arbol")

print("Numero de entradas en el arbol:", arbol.GetEntries())

# 2. Crear histogramas
hX = ROOT.TH1F("hX", "Distribucion de x; x; Eventos", 50, -5, 5)
hY = ROOT.TH1F("hY", "Distribucion de y; y; Eventos", 50, -5, 5)

# 3. Llenar histogramas recorriendo el arbol
for entry in arbol:
    hX.Fill(entry.x)
    hY.Fill(entry.y)

# 4. Dibujar resultados
c3 = ROOT.TCanvas("c3", "Ejercicio 4", 900, 400)
c3.Divide(2, 1)

c3.cd(1)
hX.Draw()

c3.cd(2)
hY.Draw()

c3.Draw()

# 5. (Opcional) Crear un TGraph de y vs x
g_xy = ROOT.TGraph(arbol.GetEntries())
for i, entry in enumerate(arbol):
    g_xy.SetPoint(i, entry.x, entry.y)

c4 = ROOT.TCanvas("c4", "Correlacion y vs x", 600, 600)
g_xy.SetTitle("y vs x; x; y")
g_xy.Draw("AP")
c4.Draw()

---

## Ejercicio 5: Crear un `TTree` con variables físicas simuladas

**Objetivo:** Construir un árbol más realista con variables que recuerden a un análisis de física (por ejemplo, energía y carga).

### Instrucciones

1. Crea un archivo `datos_detector.root`.
2. Define un `TTree` `"eventos"` con las ramas:
   - `energy` (float): energía medida, distribuida aproximadamente como una gaussiana centrada en 50 con sigma 10.
   - `charge` (float): proporcional a la energía (`charge ≈ 0.2 * energy + ruido_gaussiano`).
3. Llena el árbol con 10 000 eventos.
4. Guarda el archivo.

Este árbol será reutilizado en ejercicios posteriores.


In [ ]:
# Ejercicio 5

from array import array
import random

# 1. Crear archivo ROOT
f_det = ROOT.TFile("datos_detector.root", "RECREATE")

# 2. Crear TTree y ramas
eventos = ROOT.TTree("eventos", "Datos simulados de un detector")

energy = array('f', [0.])
charge = array('f', [0.])

eventos.Branch("energy", energy, "energy/F")
eventos.Branch("charge", charge, "charge/F")

# 3. Llenar con 10 000 eventos
for i in range(10000):
    energy[0] = random.gauss(50, 10)
    charge[0] = 0.2 * energy[0] + random.gauss(0, 2)
    eventos.Fill()

# 4. Guardar
eventos.Write()
f_det.Close()

print("Archivo 'datos_detector.root' creado con el TTree 'eventos'.")

---

## Ejercicio 6: Aplicar cortes y comparar distribuciones

**Objetivo:** Practicar la aplicación de cortes (selecciones) sobre los datos almacenados en un `TTree`.

### Instrucciones

1. Abre el archivo `datos_detector.root` y obtén el árbol `"eventos"`.
2. Construye dos histogramas de `energy`:
   - Uno sin cortes.
   - Otro con el corte `energy > 40`.
3. Dibuja ambos histogramas en el mismo `TCanvas` para compararlos.

Puedes usar:
- Un bucle y una condición `if` por evento, o
- Directamente `tree.Draw("energy>>histo", "energy>40")`.


In [ ]:
# Ejercicio 6

# 1. Abrir archivo y obtener TTree
f_det_in = ROOT.TFile("datos_detector.root", "READ")
tree_det = f_det_in.Get("eventos")

# 2. Crear histogramas
hE_all = ROOT.TH1F("hE_all", "Energia (todos los eventos); E [unidades]; Eventos", 50, 0, 100)
hE_cut = ROOT.TH1F("hE_cut", "Energia (E > 40); E [unidades]; Eventos", 50, 0, 100)

# 3. Llenar histogramas recorriendo el arbol
for entry in tree_det:
    hE_all.Fill(entry.energy)
    if entry.energy > 40:
        hE_cut.Fill(entry.energy)

# 4. Dibujar ambos en el mismo canvas
c5 = ROOT.TCanvas("c5", "Ejercicio 6 - Cortes", 800, 600)

hE_all.SetLineColor(ROOT.kBlue)
hE_cut.SetLineColor(ROOT.kRed)

hE_all.Draw("HIST")
hE_cut.Draw("HIST SAME")

c5.BuildLegend()
c5.Draw()

---

## Ejercicio 7: Uso de `RDataFrame` con datos de detector

**Objetivo:** Introducir `RDataFrame` como herramienta de alto nivel para el análisis de datos.

### Instrucciones

1. Crea un `RDataFrame` a partir del árbol `"eventos"` en `datos_detector.root`.
2. Calcula:
   - El número de eventos.
   - El valor medio (`Mean`) de `energy` y `charge`.
   - La desviación estándar (`StdDev`) de `energy`.
3. Crea un histograma de `energy` aplicando el filtro `energy > 45`.

Utiliza métodos como `Filter`, `Histo1D`, `Mean`, `StdDev`.


In [ ]:
# Ejercicio 7

# 1. Crear RDataFrame desde el archivo y el TTree
df = ROOT.RDataFrame("eventos", "datos_detector.root")

# 2. Calcular cantidades globales
n_events = df.Count()
mean_energy = df.Mean("energy")
mean_charge = df.Mean("charge")
std_energy = df.StdDev("energy")

print("Numero de eventos =", int(n_events.GetValue()))
print("Media de energy   =", float(mean_energy.GetValue()))
print("Media de charge   =", float(mean_charge.GetValue()))
print("Sigma de energy   =", float(std_energy.GetValue()))

# 3. Aplicar un filtro energy > 45
df_cut = df.Filter("energy > 45", "Corte en energia")

# 4. Crear histograma de energy con el filtro
hE_df = df_cut.Histo1D(
    ("hE_df", "Energia con energy > 45; E [unidades]; Eventos", 50, 0, 100),
    "energy"
)

c6 = ROOT.TCanvas("c6", "Ejercicio 7 - RDataFrame", 800, 600)
hE_df.Draw()
c6.Draw()

---

## Ejercicio 8: Histograma 2D (`TH2F`) de energía vs. carga

**Objetivo:** Visualizar la correlación entre dos variables a través de un histograma bidimensional.

### Instrucciones

1. A partir del árbol `"eventos"` del archivo `datos_detector.root`, crea un `TH2F` con:
   - Eje X: `energy` (0 a 100).
   - Eje Y: `charge` (por ejemplo, -10 a 40, ajusta si es necesario).
2. Llena el histograma recorriendo el árbol.
3. Dibuja el histograma en modo `"COLZ"` para visualizar el mapa de densidad.

Reflexiona: ¿se observa una correlación aproximadamente lineal entre `energy` y `charge`?


In [ ]:
# Ejercicio 8

# 1. Abrir archivo y obtener TTree (si no está ya abierto)
f_det_in2 = ROOT.TFile("datos_detector.root", "READ")
tree_det2 = f_det_in2.Get("eventos")

# 2. Crear histograma 2D
h2 = ROOT.TH2F("h2", "Energy vs Charge; Energy [unidades]; Charge [unidades]", 
               50, 0, 100, 50, -10, 40)

# 3. Llenar histograma
for entry in tree_det2:
    h2.Fill(entry.energy, entry.charge)

# 4. Dibujar
c7 = ROOT.TCanvas("c7", "Ejercicio 8 - TH2F", 700, 600)
h2.Draw("COLZ")
c7.Draw()

---

## Ejercicio 9: Guardar resultados de análisis en un archivo ROOT

**Objetivo:** Aprender a guardar múltiples objetos (histogramas, etc.) en un archivo ROOT para uso posterior.

### Instrucciones

1. Crea un archivo `resultados_analisis.root` en modo `"RECREATE"`.
2. Guarda en ese archivo:
   - El histograma `hE_all` (Ejercicio 6).
   - El histograma `hE_cut` (Ejercicio 6).
   - El histograma 2D `h2` (Ejercicio 8).
3. Cierra el archivo.
4. (Opcional) En una nueva celda, vuelve a abrir el archivo y verifica que puedes recuperar y dibujar estos histogramas.

Este patrón es típico en análisis reales, donde los resultados intermedios se almacenan para ser reutilizados.


In [ ]:
# Ejercicio 9

# 1. Crear archivo de salida
f_res = ROOT.TFile("resultados_analisis.root", "RECREATE")

# 2. Escribir objetos (asegúrate de que existan en memoria)
hE_all.Write()
hE_cut.Write()
h2.Write()

# 3. Cerrar archivo
f_res.Close()

print("Archivo 'resultados_analisis.root' creado con histogramas de energia y el TH2F energy vs charge.")

---

## Comentarios finales

En esta práctica has trabajado con:

- Histogramas 1D y 2D para explorar distribuciones y correlaciones.
- Árboles (`TTree`) para almacenar grandes cantidades de eventos.
- Archivos ROOT (`TFile`) para guardar y recuperar datos.
- `RDataFrame` como interfaz de análisis de alto nivel.

Preguntas para reflexionar:

1. ¿Qué ventajas tiene usar ROOT frente a otras herramientas de análisis más genéricas como solo NumPy/Pandas/Matplotlib?
2. ¿Cómo cambiaría esta práctica si en lugar de datos simulados usaras datos reales de un experimento?
3. ¿Qué pasos añadirías para hacer un análisis más completo (por ejemplo, ajustes, estimación de parámetros físicos, etc.)?

Puedes extender este notebook añadiendo:

- Ajustes (fits) sobre los histogramas de energía.
- Cortes más complejos (por ejemplo, en energía y carga simultáneamente).
- Nuevas variables derivadas (como funciones de `energy` y `charge`).